# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_data = pd.read_csv("C:/Users/kwelter/Desktop/Repositories/python_APIs/output_data/cities.csv") 
city_weather_data

,Unnamed: 0,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,Tezu,27.92,96.17,79.16,85,100,1.92,IN,1595681505
1,1,Saldanha,-33.01,17.94,66.20,59,0,9.17,ZA,1595681505
2,2,Bluff,-46.60,168.33,48.94,82,23,18.59,NZ,1595681506
3,3,Bredasdorp,-34.53,20.04,71.60,56,0,8.05,ZA,1595681506
4,4,Caravelas,-17.71,-39.25,73.44,86,100,5.21,BR,1595681506
...,...,...,...,...,...,...,...,...,...,...
550,550,Lufilufi,-13.87,-171.60,81.59,72,21,21.97,WS,1595681608
551,551,Opuwo,-18.06,13.84,77.36,18,0,10.42,NaN,1595681608
552,552,Ndendé,-2.40,11.36,92.73,37,22,2.66,GA,1595681608
553,553,Magadan,59.57,150.80,51.58,92,14,5.79,RU,1595681609


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Lat and Lon as Locations
locations = city_weather_data[['Lat', 'Lon']].astype(float)
#Humidity as Weight
humidity = city_weather_data['Humidity'].astype(float)

In [5]:
#Heatmap
fig = gmaps.figure(zoom_level = 2, center = (0, 0))


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                dissipating=False, max_intensity=100,
                                 point_radius = 1)


fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Maximum temperature lower than 85 degrees but higher than 70.
max_temp_df = city_weather_data.loc[(city_weather_data['Max_Temp'] >70) &(city_weather_data['Max_Temp'] <85)]

#Wind speed less than 5 mph.
max_temp_wind_df = max_temp_df.loc[(city_weather_data['Wind_Speed'] <5)]

#Cloudiness less than 1
max_temp_wind_noclouds_df = max_temp_wind_df.loc[(city_weather_data['Cloudiness'] <1)]

#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
max_temp_wind_noclouds_df

,Unnamed: 0,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
16,16,Ponta do Sol,32.67,-17.10,73.99,69,0,2.15,PT,1595681508
73,73,Santa Marinella,42.03,11.85,84.99,38,0,1.99,IT,1595681521
183,183,Kyzyl-Suu,42.34,78.00,72.45,45,0,3.78,KG,1595681540
222,222,Sampit,-2.53,112.95,75.13,85,0,4.72,ID,1595681547
231,231,Pervomayskiy,53.25,40.29,70.23,46,0,3.27,RU,1595681549
406,406,Vila Velha,-20.33,-40.29,75.99,73,0,4.70,BR,1595681582
475,475,Mozarlândia,-14.74,-50.57,84.65,32,0,4.88,BR,1595681594
527,527,Prainha,-1.80,-53.48,83.75,65,0,4.27,BR,1595681604
545,545,Linhares,-19.39,-40.07,79.43,59,0,1.03,BR,1595681607


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
max_temp_wind_noclouds_df["Hotel Name"] = ""
hotel_df = max_temp_wind_noclouds_df


C:\Users\kwelter\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
params = {
    "type": "hotel",
    "radius": 5000,
    "key": g_key
}

for index, row in max_temp_wind_noclouds_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    
    params["location"] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_data = requests.get(base_url, params=params)
    

    hotel_data = hotel_data.json()
    #print(json.dumps(hotel_data, indent= 4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing")

In [8]:
# Save Data to csv
hotel_df.to_csv("cities.csv")

# Visualize to confirm airport data appears
hotel_df.head()

,Unnamed: 0,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
16,16,Ponta do Sol,32.67,-17.10,73.99,69,0,2.15,PT,1595681508,
73,73,Santa Marinella,42.03,11.85,84.99,38,0,1.99,IT,1595681521,
183,183,Kyzyl-Suu,42.34,78.00,72.45,45,0,3.78,KG,1595681540,
222,222,Sampit,-2.53,112.95,75.13,85,0,4.72,ID,1595681547,
231,231,Pervomayskiy,53.25,40.29,70.23,46,0,3.27,RU,1595681549,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [10]:

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)




# Display figure
fig

Figure(layout=FigureLayout(height='420px'))